# Lucas Kanade Optical Flow

## Motivation
* Optical flow is the pattern of motion of objects, surfaces, and edges in a visual scene caused by the relative motion between an observer and a scene
* It is a 2D vector field where each vector is a displacement vector showing the movement of points from first frame to second

## Applications
Optical flow has many application domains:
* Structure from motion
* Video compression
* Video stabilization
* Autonomous vehicles
* Object detection and tracking

## Foundational Ideas
It works on several assumptions:
* The pixel intensities of an object do not change between consecutive frames
* Neighbouring pixels have similar motion
* It assumes that motion is smooth, which means that the motion of one object is not independent of the motion of its neighbours

# Images as 3D objects
* Previously we've talked about how although images are 2D arrays of pixels, they are better thought of as samples from a continuous image surface: $I(x, y)$
* We've also talked about other dimensions that we could consider such as
    * Scale
    * Color
    * Time
* For now, we'll focus on the time dimension
* This means we can think of a sequence of frames (video) as samples of a 3D function: $I(x, y, t)$

# Optical Flow Foundational Equation
* Suppose the frames are capturing a moving object
* This means from one frame to another, the image will be displaced by some amount
    * We considered this idea previously in the context of Harris corner detection
* The object is also moving through the frame over time
* Suppose our object moves $(dx, dy)$ in a time interval of $dt$
* Assuming the intensity doesn't change and the movement is purely translational, we can write the following equation:
\begin{equation*}
    I(x + dx, y + dy, t + dt) = I(x, y, t) 
    \hspace{100em}
\end{equation*}
If we then take the Taylor expansion of the right hand side, we get:
\begin{align*}
    I(x + dx, y + dy, t + dt) &\approx I(x, y, t) + \frac{\partial I}{\partial x}dx + \frac{\partial I}{\partial y}dy + \frac{\partial I}{\partial t}dt
    \hspace{100em}
\end{align*}
which means
\begin{align*}
    I(x + dx, y + dy, t + dt) &= I(x, y, t) \\
    I(x + dx, y + dy, t + dt) - I(x, y, t) &= 0 \\
    \frac{\partial I}{\partial x}dx + \frac{\partial I}{\partial y}dy + \frac{\partial I}{\partial t}dt &= 0\\
    \frac{\partial I}{\partial x} \frac{dx}{dt} + \frac{\partial I}{\partial y} \frac{dy}{dt}  + \frac{\partial I}{\partial t} &=0\\
    \frac{\partial I}{\partial x} u + \frac{\partial I}{\partial y} v + \frac{\partial I}{\partial t} &=0
    \hspace{100em}
\end{align*}
where $u \triangleq \frac{dx}{dt}$ and $v \triangleq \frac{dy}{dt}$ are the velocities in the $x$ and $y$ directions respectively


## Lucas-Kanade Method

### History
* Original paper was published in 1981!
* Ideas are still used today
* Paper was about image registration, with a focus on stereo matching for depth estimation
* The method was later adapted for optical flow estimation (1981)

### Original Paper Ideas

* Goal: Estimate the displacement of a patch between two frames
* Considered three measures of match
    * $L_1$ norm
    * $L_2$ norm
    * Cross correlation
* Argued, weirdly, that $L_1$ is an "inexpensive" approximation to the $L_2$ norm

#### Previous Ideas
- Exhaustive search
    - Limitations
        - Ridiculously expensive (computational cost)
    - If image is $N \times N$ and the search region is $M \times M$, then the cost is $\mathcal{O}(N^2M^2)$
- Hill-climbing 
    - Start with an initial guess and iteratively improve
    - Perhaps each iterative search is small (say $3 \times 3$), but you may have to do this many times
    - Limitations
        - Unclear how to choose the initial guess
        - Has a tendency to get stuck in local minima (false peaks)
    - Claimed, without evidence, requires $\mathcal{O}(M^2 N)$ operations
- Sequential Similarity Detection
    - When calculating the sum of a norm, if it exceeds the best match so far, stop
- Coarse-Fine Search
    - Basically an image pyramid
    - Start at a low resolution and work your way up
    - Low resolution match constrains the region of matches at the next level of higher resolution
- Combinations
    - Most of the above methods can be combined

#### Lucas-Kanade Key Idea In One Dimension
- Perhaps best understood with one-dimensional example
- Suppose we have $g(x)$ and wish to match it to $f(x + h)$ for some $h$
- We can use the Taylor expansion to approximate $f(x + h)$
\begin{align*}
    f(x + h) &\approx f(x) + f'(x)h + \ldots
    \hspace{100em}
\end{align*}
- We can then minimize the $L_2$ norm of the error
\begin{align*}
    E(h) &\triangleq \sum_{x \in R} \left(f(x + h) - g(x)\right)^2 \\
    &\approx \sum_{x \in R}  \left(f(x) + f'(x)h - g(x)\right)^2 \\
    \frac{d E(h)}{d h}
    &= 2 \sum_{x \in R} \left(f(x) + f'(x)h - g(x)\right) f'(x) \\
    \sum_{x \in R} \left(f(x) + f'(x)\hat{h} - g(x)\right) f'(x) &= 0 \\
    \sum_{x \in R} f'(x) \left(f(x) + f'(x)\hat{h} - g(x)\right) &= 0 \\
    \hat{h} &= \frac{\sum_{x \in R} f'(x) \left(g(x) - f(x)\right)}{\sum_{x \in R} f'(x)^2}
    \hspace{100em}
\end{align*}




#### Lucas-Kanade Key Idea In Two Dimensions
- If the displacement is from one frame to another in a video sequence, then the displacement is effectively a velocity vector
    - It represents the change in position over the inter-frame interval $dt$    
- Let the velocities be represented as $u$ and $v$ that satisfy the optical flow equation in that window

Let's assume we wish to minize the $L_2$ norm of the error between one patch in a frame and a shifted patch in the next frame. We'll use the first order Taylor expansion to approximate the shifted patch. We'll also assume that the motion is small, so we can ignore the higher order terms. This gives us the following equation:
\begin{align*}
    I(x+u, y+v, t) &\approx I(x, y, t) + \frac{\partial I(x,y,t)}{\partial x}dx + \frac{\partial I(x,y,t)}{\partial y}dy \\
    E(h) &\triangleq \sum_{x \in R} \left(I(x+u, y+v, t) - I(x, y, t + dt)\right)^2 \\
    &\approx \sum_{x \in R}  \left(I(x, y, t) + \frac{\partial I(x,y,t)}{\partial x} u + \frac{\partial I(x,y,t)}{\partial y} v - I(x, y, t + dt)\right)^2 
    \hspace{100em}
\end{align*}
where $h \triangleq [u, v]^T$ is the vector of velocities (changes from one frame to the next). The gradient of $E(h)$ is:
\begin{align*}
    \nabla_h E(h) &\triangleq 
    \begin{bmatrix}
        \frac{\partial E}{\partial u} \\
        \frac{\partial E}{\partial v}
    \end{bmatrix} \\
    &= 2 \sum_{x,y \in R} \left(I(x, y, t) + \frac{\partial I(x,y,t)}{\partial x} u + \frac{\partial I(x,y,t)}{\partial y} v - I(x, y, t + dt)\right) \nabla_{x,y} I(x,y,t) \\
    &= 2 \sum_{x,y \in R} \nabla_{x,y} I(x,y,t) \left(I(x, y, t) + (\nabla_{x,y} I(x,y,t))^T h - I(x, y, t + dt)\right) \\
    &= 0
    \hspace{100em}
\end{align*}
which means if we let $g \triangleq \nabla_{x,y} I(x,y,t)$ represent the image gradient (column) vector, we can solve for $h$ as follows:
\begin{align*}
    \sum_{x,y \in R} g \left(I(x, y, t) + g^T h - I(x, y, t + dt)\right)  &= 0 \\
    \left(\sum_{x,y \in R} g \left(g^T \right)\right) h &= \sum_{x,y \in R} g \left(I(x, y, t + dt) - I(x, y, t)\right) \\
    h &= \left(\sum_{x,y \in R} \left(g g^T \right)^{-1}\right) \sum_{x,y \in R} g \left(I(x, y, t + dt) - I(x, y, t)\right)
    \hspace{100em}
\end{align*}

* Note that like the Harris corner detector, this is driven by the average outer product of the Image gradient
* However, unlike the Harris corner detector, we're solving for the most accurate displacement of the patch in the next frame

### Possible Improvements
- Spatial weighting
    - Typically Gaussian
- Acount for affine transforms
    - This would require a 6D vector instead of a 2D vector
- Pyramidal approach
    - This would involve solving for the optical flow at multiple scales
    - This would allow for larger displacements to be captured
- Account for changes in lighting 
    - Brightness (additive) 
    - Contrast (multiplicative)
- Do iterative improvements
    - This would involve solving for the optical flow, then warping the image and solving again
    - This would be repeated until convergence or you reach the limit of your computational budget 

For example, even in 1981, the authors suggested the following to account for affine transformations and changes in brightness and contrast:
\begin{align*}
    E(h) &\triangleq \sum_{x \in R} \left(I(xA + h) - \left(\alpha I(x, y, t + dt) + \beta\right) \right)^2 
    \hspace{100em}
\end{align*}
where $x$ here is a 2D vector representing the spatial location, $A$ is a $2 \times 2$ matrix, and $\alpha$ and $\beta$ are the brightness and contrast changes respectively
- Note that affine transformations are a generalization of translations, rotations, and scaling
- However, writing the equation and solving it are two different things
    - The authors did not propose a method for estimating $A$, $\alpha$, and $\beta$ along with $h$
    - They did note that if we "ignore A", the problem is equivalent to maximizing the correlation coefficient between the two images
        - However, this is just a problem statement, and not a solution


## From Registration to Optical Flow
* The original paper was about image registration
* In 1981, the Tomasi and Kanade wrote a technical report about using the Lucas-Kanade method for optical flow
* The idea is to use the Lucas-Kanade method to estimate the displacement of a patch from one frame to the next
* However, this leaves an open problem: which patches should you track?
* Their answer, in principle, was that good features are those that can be tracked well
* They reduced this to meaning that the registration problem can be "solved reliably"
* Which they then claimed meant that the problem was "well conditioned"
* And, finally, they then claimed means that __both of the eigenvalues__ of the gradient outer product are "large"
    * You've heard this before
    * This is the same thinking that led to the Harris corner detector
    * Oddly, they didn't cite the Harris paper
* How do you pick a threshold for the eigenvalues?
    * They measured the eigenvalues in uniform brightness
    * This producd a lower bound
        * The threshold has to be larger than this because they don't want to track "uniform" patches
    * They also measured the eigenvalues in "highly textured" regions
    * This produced an upper bound
        * The threshold has to be smaller than this because they don't want to track miss the "highly textured" patches
    * They then picked a threshold in between
        * Said the bounds are well apart
        * Said the threshold is "not critical"

In [9]:
# Apply Lucas-Kanade method to track the motion of a point in a video
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

# Read the video
cap = cv.VideoCapture('videos/vtest.avi')

# Read the first frame
ret, first_frame = cap.read()

# Convert the first frame to grayscale
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)

# Create a zero array for the mask
mask = np.zeros_like(first_frame)

# Use Harris corner detector to find 20 best corners and track all of them
corners = cv.goodFeaturesToTrack(prev_gray, 20, 0.01, 10)

# Create an array to store the previous points
prev_points = corners

# Create a mask for the drawing
mask = np.zeros_like(first_frame)

# Read the video frame by frame
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Convert the frame to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calculate the optical flow using Lucas-Kanade method
    next_points, status, error = cv.calcOpticalFlowPyrLK(prev_gray, gray, prev_points, None)

    # Select the good points
    good_new = next_points[status == 1]
    good_prev = prev_points[status == 1]

    # Draw the tracks
    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        x_new, y_new = map(int, new.ravel())
        x_prev, y_prev = map(int, prev.ravel())
        mask = cv.line(mask, (x_new, y_new), (x_prev, y_prev), (0, 255, 0), 3)
        frame = cv.circle(frame, (x_new, y_new), 8, (0, 0, 255), -1)
    img = cv.add(frame, mask)

    # Show the result
    cv.imshow('frame', img)

    # Break the loop
    if cv.waitKey(30) & 0xFF == 27:
        break

    # Update the previous frame and previous points
    prev_gray = gray.copy()
    prev_points = good_new.reshape(-1, 1, 2)
    
# Release the video capture object
cap.release()

# Close all the windows
cv.destroyAllWindows()
cv.waitKey(1) # extra waitKey sometimes needed to close the windows

-1

## Summary
- Obvious not perfect
- Features that are "good to track" may be features that don't move
- Features that are "bad to track" may be features that move a lot
- During occlusions, the features may start tracking something else

### Limitations
- Not invariant to affine transformations
- Not invariant to changes in brightness and contrast
- Lots of parameters to tune
- Designed for grayscale images (videos)
    - Doesn't use color information

## Dense Optical Flow

* Lucas-Kanade is based on tracking "features" (corners, edges, keypoints, etc.)
* Dense optical flow is based on tracking every pixel in the image
* This is computationally expensive, but can be useful in some applications
* The most common algorithm for dense optical flow is the Farneback method

## Farneback Method
* The Farneback method is based on polynomial expansion of the image
* It works by approximating the image motion with polynomial expansion
* It then solves for the polynomial coefficients that minimize the difference between the two frames
* This gives us two different types of information about each pixel
    * The displacement is a vector
    * Much like a gradient, we can think about the magnitude and angle
    * Could represent the direction (angle) as color
    * Could represent the magnitude as intensity or opacity
* It is computationally efficient and can be used for real-time applications


In [1]:
import numpy as np
import cv2 as cv
import os 

cap = cv.VideoCapture(cv.samples.findFile("videos/vtest.avi"))

ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    
    # Show the frames side by side
    two_frames = cv.resize(frame2, (640, 480))
    bgr = cv.resize(bgr, (640, 480))
    numpy_horizontal = np.hstack((two_frames, bgr))
    cv.imshow('Side By Side', numpy_horizontal)    
    
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next

cv.destroyAllWindows()
cv.waitKey(1) # Will not close window on a Mac without this line

-1